In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# データの前処理
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# MNISTデータセットのダウンロードと準備
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# CNNモデルの定義
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 畳み込み層 (1チャネルの28x28の入力画像に対して、32個の3x3のフィルターを使用)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)

        # プーリング層 (2x2の最大プーリング)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # 全結合層
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # 畳み込み結果を入力として全結合層に接続
        self.fc2 = nn.Linear(128, 10)  # 出力は10クラス（0-9）

    def forward(self, x):
        # 畳み込み -> ReLU -> プーリング
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))

        # Flatten (平坦化)
        x = x.view(-1, 64 * 7 * 7)

        # 全結合層 -> ReLU
        x = torch.relu(self.fc1(x))

        # 最終出力
        x = self.fc2(x)
        return x

# モデル、損失関数、最適化手法の設定
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練ループ
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # 勾配の初期化
        optimizer.zero_grad()

        # フォワードパス
        outputs = model(images)
        loss = criterion(outputs, labels)

        # バックプロパゲーションとパラメータの更新
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# テストループ
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 17647815.17it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 482545.04it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 1768033.57it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 12683441.26it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/1], Loss: 0.1561
Test Accuracy: 98.42%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# データの前処理 (データをテンソル化し、正規化)
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), # データ拡張として水平反転
    transforms.RandomCrop(32, padding=4), # ランダムクロップ
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # 各チャネルを正規化
])

# CIFAR-10データセットのダウンロードと準備
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# CNNモデルの定義
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 畳み込み層 (3チャネルの32x32の入力画像に対して、32個の3x3のフィルターを使用)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)

        # プーリング層 (2x2の最大プーリング)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # 全結合層
        self.fc1 = nn.Linear(128 * 4 * 4, 256)  # 畳み込み結果を入力として全結合層に接続
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)  # 出力は10クラス（0-9）

    def forward(self, x):
        # 畳み込み -> ReLU -> プーリング
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))

        # Flatten (平坦化)
        x = x.view(-1, 128 * 4 * 4)

        # 全結合層 -> ReLU
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))

        # 最終出力
        x = self.fc3(x)
        return x

# モデル、損失関数、最適化手法の設定
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練ループ
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        # 勾配の初期化
        optimizer.zero_grad()

        # フォワードパス
        outputs = model(images)
        loss = criterion(outputs, labels)

        # バックプロパゲーションとパラメータの更新
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# テストループ
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/1], Loss: 1.6037
